In [1]:
# Imports

import pandas as pd
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt

In [2]:
### reading and sampling the data

def read_file(path):
    """
    reads the file in pandas df and converts the date_time column to datetime type
    """
    df = pd.read_csv(path)
    df['date_time'] = pd.to_datetime(df['date_time'])
    return df

def sample_on_srch_id(df, frac = 0.1):
    """
    samples the dataframe based on the fraction of srach_id
    """
    # get unique srch_ids
    srch_ids = np.unique(df.srch_id)
    # calculate how many ids to return
    chosen_k = int(len(srch_ids) * frac)
    # sample ids
    chosen_ids = random.sample(list(srch_ids), k = chosen_k)
    # filter the df to only have sampled ids
    return df[df['srch_id'].isin(chosen_ids)]

### Feature Engineering --------------------------

## missing data ----------------------------------

def remove_missing_values(df):
    """
    removes columns with more than 50 percent missing data
    """
    missing_values = df.isna().mean().round(4) * 100
    missing_values = pd.DataFrame(missing_values).reset_index()
    missing_values.columns = ["column", "missing"]
    # filter where there are missing values
    missing_values.query("missing > 50", inplace=True)  # remove columns with more than 50 % of missing values
    missing_values.sort_values("missing", inplace=True)
    #print(missing_values)
    df.drop(missing_values.column, axis=1, inplace=True)

def replace_missing_values(df):
    """
    imputes missing values with -1
    """
    df.fillna(value=-1, inplace=True) 

## new features ----------------------------------

def extract_time(df):
    """ 
    month, week, day of the week and hour of search
    """
    df_datetime = pd.DatetimeIndex(df.date_time)
    df["month"] = df_datetime.month
    df["week"] = df_datetime.week
    df["day"] = df_datetime.dayofweek + 1
    df["hour"] = df_datetime.hour
    del df['date_time']

def new_historical_price(df):
    """
    'unlogs' prop_log_historical_price column
    """
    df["prop_historical_price"] = (np.e ** df.prop_log_historical_price).replace(1.0, 0)
    df.drop("prop_log_historical_price", axis=1, inplace=True)

def add_price_position(df, rank_type = "dense"):
    """
    adds hotel price position ("price_position") inside "srch_id" column
    """
    ranks = df.groupby('srch_id')['price_usd'].rank(ascending=True, method = rank_type)
    df["price_position"] = ranks


def average_numerical_features(df, group_by = ["prop_id"], columns = ["prop_starrating", "prop_review_score", "prop_location_score1", "prop_location_score2"]):
    """
    adds mean, median and standard deviation per prop_id (default) 
    for columns that are related to property (default)
    """
    # caulcate means and rename columns
    means = df.groupby(group_by)[columns].mean().reset_index()
    means.columns = [means.columns[0]] + [x + "_mean" for x in means.columns[1:]]
    # caulcate median and rename columns
    medians = df.groupby(group_by)[columns].median().reset_index()
    medians.columns = [medians.columns[0]] + [x + "_median" for x in medians.columns[1:]]
    # caulcate means and rename columns
    stds = df.groupby(group_by)[columns].std().reset_index()
    stds.columns = [stds.columns[0]] + [x + "_std" for x in stds.columns[1:]]
    ## attach aggregated data to the df
    df = pd.merge(df, means, on=group_by)
    df = pd.merge(df, medians, on=group_by)
    df = pd.merge(df, stds, on=group_by)
    return df

def add_historical_booking_click(df):
    """
    creates a column with the percentage of the prop_id booked/clicked rate overall
    """
    # there are more prop_id in the test data than in train. 
    # Maybe we could still use this but would need to impute
    # with the most common value (or something else)
    
    historical = df.groupby("prop_id")[["click_bool", "booking_bool"]].mean().reset_index()
    historical.columns = [historical.columns[0]] + [x + "_rate" for x in historical.columns[1:]]
    df = pd.merge(df, historical, on="prop_id")
    return df.sort_values("srch_id")

def join_historical_data(df, path = "hist_click_book.csv"):
    """
    joins historical data according to prop_id. 
    path - location of historical data csv file
    
    """
    to_join = pd.read_csv(path)
    joined = pd.merge(df, to_join, on="prop_id")
    return joined.sort_values("srch_id")
    
    
## other ----------------------------------

def remove_positions(df, positions = [5, 11, 17, 23]):
    """
    removes hotels with specified positions 
    (based on the fact that hotels in those positions were not as booked)
    """
    df = df[df["position"].isin(positions) == False]

def add_score(df):
    """
    adds 'score' column to the df: 5 for booked, 1 for clicked
    """
    score = []
    for book, click in zip(df.booking_bool, df.click_bool):
        if book == 1:
            score.append(5)
            continue
        if click == 1:
            score.append(1)
            continue
        else:
            score.append(0)
    df["score"] = score
    del df['booking_bool']
    del df['click_bool']

def onehot(df, cols):
    """ 
    returns a df with one-hot encoded columns (cols)
    """
    
    return pd.get_dummies(df, columns=cols)


### Feature engineering function -----------

def feature_engineering_train(df):
    
    extract_time(df)
    remove_missing_values(df)
    replace_missing_values(df)
    new_historical_price(df)
    add_price_position(df)
    average_numerical_features(df)
    add_score(df)

def feature_engineering_test(df):
    
    extract_time(df)
    remove_missing_values(df)
    replace_missing_values(df)
    new_historical_price(df)
    add_price_position(df)
    average_numerical_features(df)

In [3]:
df = pd.read_csv("data/training_set_VU_DM.csv")
# testset = pd.read_csv("data/test_set_VU_DM.csv")
df.describe()

,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,...,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
count,4.958347e+06,4.958347e+06,4.958347e+06,251866.000000,252988.000000,4.958347e+06,4.958347e+06,4.958347e+06,4.950983e+06,4.958347e+06,...,96174.000000,315348.000000,356422.000000,138515.000000,1.916654e+06,1.987503e+06,614730.000000,4.958347e+06,138390.000000,4.958347e+06
mean,1.663666e+05,9.953133e+00,1.753405e+02,3.374334,176.022659,1.739739e+02,7.007918e+04,3.180525e+00,3.777777e+00,6.346994e-01,...,17.250473,0.145969,0.083202,19.433267,-6.089936e-02,9.962752e-03,22.430384,4.474858e-02,386.283316,2.791051e-02
std,9.611223e+04,7.646890e+00,6.591625e+01,0.692519,107.254493,6.834525e+01,4.060992e+04,1.051024e+00,1.050329e+00,4.815144e-01,...,31.160313,0.578202,0.316722,54.370221,4.691723e-01,2.029142e-01,895.965854,2.067514e-01,821.190577,1.647165e-01
min,1.000000e+00,1.000000e+00,1.000000e+00,1.410000,0.000000,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.000000,-1.000000,-1.000000,2.000000,-1.000000e+00,-1.000000e+00,2.000000,0.000000e+00,0.000000,0.000000e+00
25%,8.293600e+04,5.000000e+00,1.000000e+02,2.920000,109.810000,1.000000e+02,3.501000e+04,3.000000e+00,3.500000e+00,0.000000e+00,...,6.000000,0.000000,0.000000,7.000000,0.000000e+00,0.000000e+00,7.000000,0.000000e+00,124.000000,0.000000e+00
50%,1.665070e+05,5.000000e+00,2.190000e+02,3.450000,152.240000,2.190000e+02,6.963800e+04,3.000000e+00,4.000000e+00,1.000000e+00,...,11.000000,0.000000,0.000000,12.000000,0.000000e+00,0.000000e+00,11.000000,0.000000e+00,218.400000,0.000000e+00
75%,2.497240e+05,1.400000e+01,2.190000e+02,3.930000,213.490000,2.190000e+02,1.051680e+05,4.000000e+00,4.500000e+00,1.000000e+00,...,18.000000,1.000000,0.000000,20.000000,0.000000e+00,0.000000e+00,17.000000,0.000000e+00,429.790000,0.000000e+00
max,3.327850e+05,3.400000e+01,2.310000e+02,5.000000,1958.700000,2.300000e+02,1.408210e+05,5.000000e+00,5.000000e+00,1.000000e+00,...,1620.000000,1.000000,1.000000,9900.000000,1.000000e+00,1.000000e+00,149400.000000,1.000000e+00,159292.380000,1.000000e+00


In [4]:
# feature engineering for test and trainingset

df = add_historical_booking_click(df)
feature_engineering_train(df)

# feature_engineering_test(testset)

<ipython-input-2-e260ecf04c46>:55: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  df["week"] = df_datetime.week


In [5]:
# properties = testset['prop_id']
properties = df["prop_id"]

In [6]:
del df['prop_id']
del df['position']
# del testset['prop_id']

In [7]:
# trainingset (df) should have 1 column more; the target column

# len(testset.columns), len(df.columns)

In [8]:
df.head(10)

,srch_id,site_id,visitor_location_country_id,prop_country_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,price_usd,...,random_bool,click_bool_rate,booking_bool_rate,month,week,day,hour,prop_historical_price,price_position,score
0,1,12,187,219,3,3.5,1,2.83,0.0438,104.77,...,1,0.026144,0.016340,4,14,4,8,141.174964,3.0,0
14076,1,12,187,219,3,2.5,1,0.69,0.0071,120.29,...,1,0.017391,0.008696,4,14,4,8,106.697742,5.0,0
13444,1,12,187,219,3,4.5,1,2.20,0.0164,181.09,...,1,0.023734,0.015823,4,14,4,8,151.411304,16.0,0
12851,1,12,187,219,2,3.5,1,2.71,0.0465,128.06,...,1,0.016863,0.008432,4,14,4,8,84.774942,7.0,0
12321,1,12,187,219,2,3.5,1,2.83,0.0145,115.12,...,1,0.022642,0.011321,4,14,4,8,107.770073,4.0,0
11560,1,12,187,219,4,3.5,1,2.40,0.1149,139.70,...,1,0.111695,0.093298,4,14,4,8,138.379512,10.0,0
10946,1,12,187,219,3,4.0,1,2.71,0.0359,122.88,...,1,0.042345,0.030945,4,14,4,8,111.052160,6.0,0
10313,1,12,187,219,4,4.0,1,2.08,0.0150,191.44,...,1,0.045814,0.031596,4,14,4,8,196.369875,18.0,0
9589,1,12,187,219,4,3.5,1,2.77,0.1266,115.12,...,1,0.089779,0.063536,4,14,4,8,145.474382,4.0,0
9450,1,12,187,219,3,3.0,1,1.39,0.0038,138.40,...,1,0.007194,0.007194,4,14,4,8,121.510418,9.0,0


### XGboost

In [9]:
# imports

import xgboost as xgb
from xgboost import DMatrix
from sklearn.model_selection import GroupShuffleSplit

In [10]:
# Only useful for experimenting

# split sample in 80 percent training and 20 percent test by srch_id
gss = GroupShuffleSplit(test_size=.3, n_splits=1, random_state = 7).split(df, groups=df['srch_id'])

X_train_inds, X_test_inds = next(gss)

In [11]:
train_data= df.iloc[X_train_inds]
X_train = train_data.loc[:, ~train_data.columns.isin(['srch_id','score'])]
y_train = train_data.loc[:, train_data.columns.isin(['score'])]

groups = train_data.groupby('srch_id').size().to_frame('size')['size'].to_numpy()

test_data= df.iloc[X_test_inds]

#We need to keep the id for later predictions
X_test = test_data.loc[:, ~test_data.columns.isin(['score'])]
y_test = test_data.loc[:, test_data.columns.isin(['score'])]

In [12]:
model = xgb.XGBRanker(  
    tree_method='hist',
    booster='gbtree',
    objective='rank:pairwise',
    random_state=42, 
    learning_rate=0.05,
    colsample_bytree=0.9, 
    eta=0.05, 
    max_depth=10, 
    n_estimators=110, 
    subsample=0.75,
    )

model.fit(X_train, y_train, group=groups, verbose=True)

XGBRanker(base_score=0.5, booster='gbtree', colsample_bylevel=1,
          colsample_bynode=1, colsample_bytree=0.9, eta=0.05, gamma=0,
          gpu_id=-1, importance_type='gain', interaction_constraints='',
          learning_rate=0.05, max_delta_step=0, max_depth=10,
          min_child_weight=1, missing=nan, monotone_constraints='()',
          n_estimators=110, n_jobs=8, num_parallel_tree=1, random_state=42,
          reg_alpha=0, reg_lambda=1, scale_pos_weight=None, subsample=0.75,
          tree_method='hist', validate_parameters=1, verbosity=None)

In [13]:
def predict(model, df):
    return model.predict(df.loc[:, ~df.columns.isin(['srch_id'])])

In [14]:
# this is not necessary right?

del test_data['score']

In [15]:
predictions = (test_data.groupby('srch_id')
               .apply(lambda x: predict(model, x)))

In [16]:
predictions

srch_id
8         [0.44458175, -1.1736909, 0.9777025, 0.71873766...
12        [1.5338484, -2.493196, -2.4340463, -2.4726849,...
17        [0.10270913, 0.6096008, 0.30111024, 0.38664553...
25        [2.380065, -1.5059115, -2.8656082, -0.48547158...
28        [0.77781194, 0.9279906, 1.0835915, -1.6429164,...
                                ...                        
332765    [0.67064387, 0.5056209, -2.3840253, 1.1351368,...
332768    [-2.8659294, 0.37186155, 0.3758127, -0.6688877...
332777    [0.8211826, 0.5279357, 1.0991979, 1.3342755, 1...
332781    [2.0604317, 0.74221253, -0.2955459, -0.8317869...
332784    [-0.6448176, 0.5084949, -0.0014021621, -0.2181...
Length: 59939, dtype: object

### NDCG

In [17]:
# Based on: https://dalelane.co.uk/blog/?p=3403
# you calculate the ndcg per query
# and then you take the mean of all ndcg's per query

def eval_ndcg(pred, y, ndgc_at=10):
    
    """
    calculate the ndcg@ndcg_at
    """
    
    

In [18]:
 # Create dataframe of testdata with optimal rankings
    
orig = pd.DataFrame()
orig["srch_id"] = test_data["srch_id"].copy()
orig["prop_id"] = properties.iloc[X_test_inds].copy()
orig["score"] = y_test.copy()

# Sort to get optimal order
idcg_df = orig.groupby('srch_id').apply(pd.DataFrame.sort_values, 'score', ascending=False)

In [19]:
idcg_df.head()

srch_id  prop_id  score
srch_id                               
8       27391        8    27669      5
        29721        8    73738      0
        26956        8    13252      0
        26770        8    10250      0
        29908        8    83293      0

In [20]:
# Attempt to print first 10 items in groups

count = 0
for group in idcg_df.items():
    if count == 10:
        break
        
    print(group)
    count += 1

('srch_id', srch_id         
8        27391           8
         29721           8
         26956           8
         26770           8
         29908           8
                     ...  
332784   1922444    332784
         638571     332784
         641338     332784
         641619     332784
         3300395    332784
Name: srch_id, Length: 1486721, dtype: int64)
('prop_id', srch_id         
8        27391       27669
         29721       73738
         26956       13252
         26770       10250
         29908       83293
                     ...  
332784   1922444     51585
         638571      50151
         641338     114177
         641619     115562
         3300395     78171
Name: prop_id, Length: 1486721, dtype: int64)
('score', srch_id         
8        27391      5
         29721      0
         26956      0
         26770      0
         29908      0
                   ..
332784   1922444    0
         638571     0
         641338     0
         641619     0
         

In [21]:
for i in idcg_df.items():
    print(i)

('srch_id', srch_id         
8        27391           8
         29721           8
         26956           8
         26770           8
         29908           8
                     ...  
332784   1922444    332784
         638571     332784
         641338     332784
         641619     332784
         3300395    332784
Name: srch_id, Length: 1486721, dtype: int64)
('prop_id', srch_id         
8        27391       27669
         29721       73738
         26956       13252
         26770       10250
         29908       83293
                     ...  
332784   1922444     51585
         638571      50151
         641338     114177
         641619     115562
         3300395     78171
Name: prop_id, Length: 1486721, dtype: int64)
('score', srch_id         
8        27391      5
         29721      0
         26956      0
         26770      0
         29908      0
                   ..
332784   1922444    0
         638571     0
         641338     0
         641619     0
         

In [22]:
# Because we evaluate ndcg@38

count = 0:
idcg += score  
while count < 38:
    
    count += 1

SyntaxError: invalid syntax (<ipython-input-22-da790b8f4ebd>, line 3)

In [ ]:
test_data["prop_id"] = properties.iloc[X_test_inds]

In [ ]:
test_data.head()

In [ ]:
# Sort on predicted_score output within srch_id

out = output.groupby('srch_id').apply(pd.DataFrame.sort_values, 'pred_scores', ascending=False)

In [ ]:
raise NotImplementedError("Stop running")

### Format output and Write to CSV

In [ ]:
# This could be done more efficiently

# Prepare output file
output = pd.DataFrame()
output["srch_id"] = testset["srch_id"]
output["prop_id"] = properties

# Add scores
pred_scores_list = []

for i in predictions:
    for j in i:
        pred_scores_list.append(j)      

output["pred_scores"] = pred_scores_list

In [ ]:
# Sort on predicted_score output within srch_id

out = output.groupby('srch_id').apply(pd.DataFrame.sort_values, 'pred_scores', ascending=False)

In [ ]:
del out["pred_scores"]

In [ ]:
out.head()

In [ ]:
# write to csv

out.to_csv('data/submission.csv', index=False)

In [ ]:
# Most important features

xgb.plot_importance(model)